In [80]:
from bs4 import BeautifulSoup
import time
from tqdm import tqdm_notebook
import requests
import re

In [510]:
def scrape_recipe(recipe_url):
    
    html_page = requests.get(recipe_url)
    soup = BeautifulSoup(html_page.content,'html.parser')
    time.sleep(1)
    
    recipe = {}
    
    # give the recipe an ID
    recipe['ID'] = re.search('[0-9]+', recipe_url).group()
    
    # get name of recipe
    try:
        recipe['name'] = soup.find('h1',{'id':'recipe-main-content'}).text
    except AttributeError:
        recipe['name'] = soup.find('h1',{'class':'headline heading-content'}).text
    
    # get descritpion
    try:
        recipe['description'] = ' '.join(soup.find('div',{'class':'submitter__description'}).text.split())
    except AttributeError:
        recipe['description'] = soup.find('div',{'class':'recipe-summary margin-8-bottom'}).text.replace('\n','')
    
    # get ingredients
    ingredients = soup.findAll('ul',{'ng-hide':'reloaded'})
    if ingredients:
        for i in range(0,len(ingredients)):
            recipe[f'ingredients_{i+1}'] = ' '.join(ingredients[i].text.split())
    else:
        recipe['ingredients_1'] = ' '.join(soup.find('ul',{'class':'ingredients-section'}).text.replace('\n','').split())
    
    # get prep time
    try:
        recipe['prep_time'] = soup.find('time',{'itemprop':'prepTime'}).text
    except AttributeError:
        recipe_time = soup.findAll('div',{'class':'recipe-meta-item-body'})
        recipe['prep_time'] = recipe_time[0].text.strip()
    
    # get cook time
    try:
        recipe['cook_time'] = soup.find('time',{'itemprop':'cookTime'}).text
    except AttributeError:
        recipe['cook_time'] = recipe_time[1].text.strip()
        
    # get total time
    try:
        recipe['total_time'] = soup.find('time',{'itemprop':'totalTime'}).text
    except AttributeError:
        recipe['total_time'] = recipe_time[2].text.strip()
        
    # get instructions
    instructions = soup.findAll('li',{'class':'step'})
    if instructions:
        for i in range(0,len(instructions)-1):
            recipe[f'intsructions_step_{i+1}'] = instructions[i].text.replace('\n','').strip().replace('Advertisement',"")
            # check if scraping grabbed the advertisement string and drop
    else:
        instructions = soup.find('ul',{'class':'instructions-section'}).text.replace('\n','').split('Step')
        for i in range(1,len(instructions)):
            recipe[f'instructions_step_{i}'] = instructions[i].replace(f'{i}','').strip().replace('Advertisement',"")
             # check if scraping grabbed the advertisement string and drop
    
    # nutrition info
    try:
        recipe['calories'] = soup.find('span',{'itemprop':'calories'}).text.split()[0]
        recipe['fat (g)'] = soup.find('span',{'itemprop':'fatContent'}).text
        recipe['carbs (g)'] = soup.find('span',{'itemprop':'carbohydrateContent'}).text
        recipe['protein (g)'] = soup.find('span',{'itemprop':'proteinContent'}).text
    except AttributeError:
        nutrition = soup.findAll('div',{'class':'section-body'})[-1].text
        recipe['calories'] = nutrition.split()[0]
        recipe['fat (g)'] = nutrition.split()[2]
        recipe['carbs (g)'] = nutrition.split()[12]
        recipe['protein (g)'] = nutrition.split()[15]
        
    # append to master list
    return recipe

In [ ]:
def scrape_reviews(soup):
    soup.find

In [590]:
all_recipes = []
def get_recipes(url):
    for i in tqdm_notebook(range(0,64)):
        # iterate through each page
        url = url+f'?page={i}'
        recipe_urls = []
        html_page = requests.get(url)
        soup = BeautifulSoup(html_page.content,'html.parser')
        time.sleep(1)
        # on each page, grab the urls for each recipce
        for url in soup.findAll('h3', {'class':'fixed-recipe-card__h3'}):
            recipe_urls.append(url.find('a')['href'])
        for recipe in recipe_urls:
            all_recipes.append(scrape_recipe(recipe))
    return True

In [589]:
get_recipes('https://www.allrecipes.com/recipes/17562/dinner/')

KeyboardInterrupt: 

In [ ]:
# get reviews from users

# need selenium to click 'more reviews'
# then use bs4 again to grab the remaining reviews

In [ ]:
# use f stirng to go through each page 
# https://www.allrecipes.com/?page=3

In [ ]:
import selenium
from selenium import webdriver

In [ ]:
driver.get(url)

In [594]:
html_page = requests.get('https://www.allrecipes.com/recipe/220059/chef-johns-classic-beef-stroganoff/')
soup = BeautifulSoup(html_page.content,'html.parser')

In [596]:
soup.find('div',{'class':'review-container clearfix'})

'\n\nMost helpful positive review\n\n\n\n\n\n\n\n\nSarah Jo\r\n        \n\n\n\r\n                1\r\n            \n\n\n\n\r\n                5k\r\n            \n\n\n\n\n\n\n\n\n\n\n7/20/2012\n\n\n\r\nI cannot afford beef chuck roast right now, I used beef stew meat instead. I also did not have creme fraiche, I used sour cream instead. You could not tell that I used beef stew meat, the meat t...\r\n        \n\n\n\nRead more\n\n\n'